In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_data = 5000   # number of data
num_vars = 5      # number of decision variables
num_ints = 5      # number of integer decision variables
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_train = np.random.uniform(2.0, 6.0, (train_size, 1)).astype(np.float32)
a_train = np.random.uniform(6.0, 15.0, (train_size, num_vars)).astype(np.float32)
p_test = np.random.uniform(6.0, 15.0, (test_size, 1)).astype(np.float32)
a_test = np.random.uniform(0.2, 1.2, (test_size, num_vars)).astype(np.float32)
p_dev = np.random.uniform(6.0, 15.0, (val_size, 1)).astype(np.float32)
a_dev = np.random.uniform(0.2, 1.2, (val_size, num_vars)).astype(np.float32)

In [5]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train, "a":a_train}, name="train")
data_test = DictDataset({"p":p_test, "a":a_test}, name="test")
data_dev = DictDataset({"p":p_dev, "a":a_test}, name="dev")

In [6]:
# torch dataloaders
from torch.utils.data import DataLoader
loader_train = DataLoader(data_train, batch_size=32, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size=32, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size=32, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=True)

## NM Problem

In [7]:
import neuromancer as nm
from problem.neural import probRastrigin

def getNMProb(round_module):
    # parameters
    p = nm.constraint.variable("p")
    a = nm.constraint.variable("a")
    # variables
    x_bar = nm.constraint.variable("x_bar")
    x_rnd = nm.constraint.variable("x_rnd")

    # model
    obj_bar, constrs_bar = probRastrigin(x_bar, p, a, num_vars=num_vars, alpha=100)
    obj_rnd, constrs_rnd = probRastrigin(x_rnd, p, a, num_vars=num_vars, alpha=100)

    # define neural architecture for the solution mapping
    func = nm.modules.blocks.MLP(insize=num_vars+1, outsize=num_vars, bias=True,
                                 linear_map=nm.slim.maps["linear"], nonlin=nn.ReLU, hsizes=[80]*4)
    # solution map from model parameters: sol_map(p) -> x
    sol_map = nm.system.Node(func, ["p", "a"], ["x_bar"], name="smap")

    # penalty loss for mapping
    components = [sol_map]
    loss = nm.loss.PenaltyLoss(obj_bar, constrs_bar)
    problem = nm.problem.Problem(components, loss)

    # penalty loss for rounding
    components = [sol_map, round_module]
    loss = nm.loss.PenaltyLoss(obj_rnd, constrs_rnd)
    problem_rnd = nm.problem.Problem(components, loss)

    return problem, problem_rnd

## Exact Solver

In [8]:
from problem.solver import exactRastrigin
model = exactRastrigin(n_vars=num_vars, n_integers=num_ints)

In [ ]:
sols, objvals, conviols, elapseds = [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    model.setParamValue(p, *a)
    tick = time.time()
    xval, objval = model.solve("scip")
    tock = time.time()
    sols.append(list(xval.values()))
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

  4%|███▎                                                                            | 42/1000 [00:04<01:30, 10.59it/s]

In [ ]:
df

## Heuristic

In [ ]:
from heuristic import naive_round

In [ ]:
# relaxed model
model_rel = model.relax()

In [ ]:
sols, objvals, conviols, elapseds = [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    model_rel.setParamValue(p, *a)
    tick = time.time()
    xval_init, _ = model_rel.solve("scip", max_iter=100)
    naive_round(xval_init, model)
    tock = time.time()
    xval, objval = model.getVal()
    sols.append(list(xval.values()))
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

## Learning to Round

In [ ]:
from model.layer import netFC
from model.round import roundModel
# round x
layers_rnd = netFC(input_dim=num_vars*2+1, hidden_dims=[80]*4, output_dim=num_vars)
round_func = roundModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x_bar"], output_keys=["x_rnd"],
                        int_ind={"x_bar":model.int_ind}, name="round")
_, problem = getNMProb(round_func)

In [ ]:
# training
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

In [ ]:
sols, objvals, conviols, elapseds = [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), "a":torch.tensor(np.array([a]), dtype=torch.float32), "name": "test"}
    tick = time.time()
    output = problem(datapoints)
    tock = time.time()
    x = output["test_x_rnd"]
    # get values
    for ind in model.x:
        model.x[ind].value = x[0, ind].item()
    xval, objval = model.getVal()
    sols.append(xval.values())
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

## Learnable Threshold

In [ ]:
from model.layer import netFC
from model.threshold import roundThresholdModel
# round x
layers_rnd = netFC(input_dim=num_vars*2+1, hidden_dims=[80]*4, output_dim=num_vars)
round_func = roundThresholdModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x_bar"], output_keys=["x_rnd"],
                                 int_ind={"x_bar":model.int_ind}, name="round")
_, problem = getNMProb(round_func)

In [ ]:
# training
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

In [ ]:
sols, objvals, conviols, elapseds = [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), "a":torch.tensor(np.array([a]), dtype=torch.float32), "name": "test"}
    tick = time.time()
    output = problem(datapoints)
    tock = time.time()
    x = output["test_x_rnd"]
    # get values
    for ind in model.x:
        model.x[ind].value = x[0, ind].item()
    xval, objval = model.getVal()
    sols.append(xval.values())
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

## Learning to Round with Fixed Solution Mapping

In [ ]:
from model.layer import netFC
from model.round import roundModel
# round x
layers_rnd = netFC(input_dim=num_vars*2, hidden_dims=[80]*4, output_dim=num_vars)
round_func = roundModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x_bar"], output_keys=["x_rnd"],
                        int_ind={"x_bar":model.int_ind}, name="round")
problem_rel, problem_rnd = getNMProb(round_func)

In [ ]:
# training for mapping
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem_rel.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem_rel, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

In [ ]:
# freeze sol mapping
#problem_rel.freeze()
# training for rounding
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem_rnd, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

In [ ]:
sols, objvals, conviols, elapseds = [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), "a":torch.tensor(np.array([a]), dtype=torch.float32), "name": "test"}
    tick = time.time()
    output = problem_rnd(datapoints)
    tock = time.time()
    x = output["test_x_rnd"]
    # get values
    for ind in model.x:
        model.x[ind].value = x[0, ind].item()
    xval, objval = model.getVal()
    sols.append(xval.values())
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

### Learning to Feasibility Pump

In [ ]:
from model.layer import netFC
# define neural architecture for the solution mapping
sol_func = nm.modules.blocks.MLP(insize=num_vars+1, outsize=num_vars, bias=True,
                                 linear_map=nm.slim.maps["linear"], nonlin=nn.ReLU,
                                 hsizes=[80]*4)
# define neural architecture for rounding
rnd_layer = netFC(input_dim=num_vars*2+1, hidden_dims=[80]*4, output_dim=num_vars)

In [ ]:
# function to get nm optimization model
from problem.neural import probRastrigin
def getProb(x, p, a):
    return probRastrigin(x, p, a, num_vars=num_vars, alpha=100)

In [ ]:
from model import feasibilityPumpModel
# feasibility pump model
problem_rel, problem_fp = feasibilityPumpModel(["p", "a"], getProb, sol_func, rnd_layer, int_ind=model.int_ind, num_iters=3)

In [ ]:
# training for mapping
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem_rel.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem_rel, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

In [ ]:
# freeze sol mapping
#problem_rel.freeze()
# training for feasibility pump
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem_fp, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

In [ ]:
sols, objvals, conviols, elapseds = [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), "a":torch.tensor(np.array([a]), dtype=torch.float32), "name": "test"}
    tick = time.time()
    output = problem(datapoints)
    tock = time.time()
    x = output["test_x_rnd"]
    # get values
    for ind in model.x:
        model.x[ind].value = x[0, ind].item()
    xval, objval = model.getVal()
    sols.append(xval.values())
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())